In [2]:
from langgraph.graph import StateGraph, START, END
from langgraph.types import Command, interrupt
from typing import TypedDict
from langgraph.checkpoint.memory import MemorySaver

In [20]:
memory = MemorySaver()
class State(TypedDict):
    value: str

In [21]:
def node_a(state: State):
    print("Node A")
    return Command(
        goto="node_b",
        update={
            "value": state["value"] + "a"
        }
    )

In [22]:
def node_b(state: State):
    print("Node B")
    human_reponse = interrupt("Do you want to go to C or D? Typed C/D")
    print("Human Review Value:", human_reponse)
    
    if (human_reponse == "C"):
        return Command(
            goto="node_c",
            update={
                "value": state["value"] + "c"
            }
        )
    elif (human_reponse == "D"):
        return Command(
            goto="node_d",
            update={
                "value": state["value"] + "d"
            }
        )

In [ ]:
def node_c(state: State):
    print("Node C")
    return Command(
        goto=END,
        # update={
        #     "value": state["value"] + "c"
        # }
    )
    
def node_d(state: State):
    print("Node D")
    return Command(
        goto=END,
        update={
            "value": state["value"] + "d"
        }
    )

In [24]:
graph = StateGraph(State)

graph.add_node("node_a", node_a)
graph.add_node("node_b", node_b)
graph.add_node("node_c", node_c)
graph.add_node("node_d", node_d)

graph.set_entry_point("node_a")

app = graph.compile(checkpointer=memory)

In [25]:
config = {
    "configurable": {
        "thread_id": "1"
        }   
    }

initiaState = {
    "value": ""
}

In [26]:
print(app.get_state(config).next)

()


In [ ]:
first_result = app.invoke(initiaState, config, stream_mode="updates")
first_result

Node A
Node B


[{'node_a': {'value': 'a'}},
 {'__interrupt__': (Interrupt(value='Do you want to go to C or D? Typed C/D', resumable=True, ns=['node_b:16338e4e-9d58-37d7-2a4d-f40906a306e9']),)}]

In [29]:
print(app.get_state(config).next)

('node_b',)


In [30]:
second_result = app.invoke(Command(resume="C"), config=config, stream_mode="updates")
second_result

Node B
Human Review Value: C
Node C


[{'node_b': {'value': 'ac'}}, {'node_c': {'value': 'acc'}}]

In [31]:
print(app.get_state(config).next)

()
